In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### READ IN DATA

In [4]:
chamau_lag = pd.read_csv("datasets/Chamau_Preprocessed_2014-2024.csv")
chamau_daily = pd.read_csv("datasets/Chamau_Daily_2014-2024.csv")

chamau_A = chamau_lag[chamau_lag["Parcel"] == "A"].copy()
chamau_B = chamau_lag[chamau_lag["Parcel"] == "B"].copy()

chamau_daily_A = chamau_daily[chamau_daily["Parcel"] == "A"].set_index("Date").sort_index()
chamau_daily_B = chamau_daily[chamau_daily["Parcel"] == "B"].set_index("Date").sort_index()


In [10]:
# --- Define predictor groups ---
predictor_sets = {
    "base": [
        "Precipitation", "SolarRadiation", "AirTemp", "VPD",
        "SoilWater_5cm", "SoilWater_15cm",
        "SoilTemp_4cm", "SoilTemp_15cm",
        "NEE", "GPP", "RECO",
        "Mowing", "FertilizerOrganic", "FertilizerMineral",
        "Grazing", "SoilCultivation"
    ],

    "lag": [
        "Precipitation", "SolarRadiation", "AirTemp", "VPD",
        "SoilWater_5cm", "SoilWater_15cm",
        "SoilTemp_4cm", "SoilTemp_15cm",
        "NEE", "GPP", "RECO",

        # Meteorological lag variables
        "Precipitation_lag1d", "Precipitation_lag3d", "Precipitation_lag5d", "Precipitation_lag7d",
        "SolarRadiation_lag1d", "SolarRadiation_lag3d", "SolarRadiation_lag5d", "SolarRadiation_lag7d",
        "AirTemp_lag1d", "AirTemp_lag3d", "AirTemp_lag5d", "AirTemp_lag7d",
        "VPD_lag1d", "VPD_lag3d", "VPD_lag5d", "VPD_lag7d",
        "SoilWater_5cm_lag1d", "SoilWater_5cm_lag3d", "SoilWater_5cm_lag5d", "SoilWater_5cm_lag7d",
        "SoilWater_15cm_lag1d", "SoilWater_15cm_lag3d", "SoilWater_15cm_lag5d", "SoilWater_15cm_lag7d",
        "SoilTemp_4cm_lag1d", "SoilTemp_4cm_lag3d", "SoilTemp_4cm_lag5d", "SoilTemp_4cm_lag7d",
        "SoilTemp_15cm_lag1d", "SoilTemp_15cm_lag3d", "SoilTemp_15cm_lag5d", "SoilTemp_15cm_lag7d",
        "NEE_lag1d", "NEE_lag3d", "NEE_lag5d", "NEE_lag7d",
        "GPP_lag1d", "GPP_lag3d", "GPP_lag5d", "GPP_lag7d",
        "RECO_lag1d", "RECO_lag3d", "RECO_lag5d", "RECO_lag7d",

        # Management and days since management
        "Mowing", "FertilizerOrganic", "FertilizerMineral", "Grazing", "SoilCultivation",
        "DaysSince_Mowing", "DaysSince_FertilizerOrganic", "DaysSince_FertilizerMineral",
        "DaysSince_Grazing", "DaysSince_SoilCultivation"
    ],

    "main": [
        "AirTemp", "VPD",
        "SoilTemp_4cm", "SoilTemp_15cm",
        "RECO", "SolarRadiation"
    ]
}

# --- Define datasets (includes split daily A/B) ---
datasets = {
    "Hourly": chamau_lag,
    "Hourly A": chamau_A,
    "Hourly B": chamau_B,
    "Daily A": chamau_daily_A,
    "Daily B": chamau_daily_B,
    "Daily": chamau_daily
}

targets = ["N2O_Flux", "N2O_Flux_ln"]


In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

def train_rf_timeseries(df, predictors, target,
                        test_size=0.3, n_estimators=300, random_state=42, plot=True):
    """
    Train a Random Forest on time-series data using a chronological split.
    """

    # --- ensure time order ---
    df = df.sort_index().copy()

    # --- extract features/target ---
    X = df[predictors]
    y = df[target]

    # --- handle NaNs ---
    mask = X.notna().all(axis=1) & y.notna()
    X, y = X[mask], y[mask]

    # --- offset for positivity ---
    offset = abs(y.min()) + 1e-6
    y_shifted = y + offset

    # --- chronological split ---
    split_idx = int(len(X) * (1 - test_size))
    X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
    y_train, y_test = y_shifted.iloc[:split_idx], y_shifted.iloc[split_idx:]


    time_train = (df.index.min(), df.index[:split_idx].max())
    time_test  = (df.index[split_idx], df.index.max())
    
    print(f"Train period: {time_train[0]} → {time_train[1]}")
    print(f"Test  period: {time_test[0]} → {time_test[1]}")

    # --- train model ---
    rf = RandomForestRegressor(n_estimators=n_estimators, random_state=random_state)
    rf.fit(X_train, y_train)

    # --- predictions ---
    y_pred = rf.predict(X_test)

    # revert offset
    y_pred_lin = y_pred - offset
    y_test_lin = y_test - offset

    # --- evaluation ---
    r2 = r2_score(y_test_lin, y_pred_lin)
    rho, _ = spearmanr(y_test_lin, y_pred_lin)

    # --- feature importance ---
    importances = pd.Series(rf.feature_importances_, index=predictors).sort_values(ascending=False)

    print("\nModel evaluation:")
    print(f"  R² (linear scale): {r2:.3f}")
    print(f"  Spearman ρ:        {rho:.3f}")

    # --- plot ---
    if plot:
        plt.figure(figsize=(6, 5))
        plt.scatter(y_test_lin, y_pred_lin, alpha=0.6)
        min_val, max_val = y_test_lin.min(), y_test_lin.max()
        plt.plot([min_val, max_val], [min_val, max_val], "r--")
        plt.xlabel("Observed N₂O Flux")
        plt.ylabel("Predicted N₂O Flux")
        plt.title(f"Chronological RF (R²={r2:.2f}, ρ={rho:.2f})")
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

    return {
        "model": rf,
        "r2": r2,
        "spearman_rho": rho,
        "feature_importance": importances,
        "y_test": y_test_lin,
        "y_pred": y_pred_lin,
    }


In [24]:
# --- Run training loop ---
results_summary = []

for df_name, df in datasets.items():
    for target in targets:
        for set_name, predictors in predictor_sets.items():
            print(f"\n--- Training on {df_name} | target={target} | predictors={set_name} ---")
            try:
                res = train_rf_timeseries(
                    df=df,
                    predictors=predictors,
                    target=target,
                    plot=False  # suppress plotting
                )
                results_summary.append({
                    "Dataset": df_name,
                    "Target": target,
                    "Predictor_Set": set_name,
                    "R²": res["r2"],
                    "Spearman_ρ": res["spearman_rho"]
                })
            except Exception as e:
                print(f"Skipped {df_name} | {target} | {set_name} due to error: {e}")
                results_summary.append({
                    "Dataset": df_name,
                    "Target": target,
                    "Predictor_Set": set_name,
                    "R²": None,
                    "Spearman_ρ": None
                })

# --- Combine results ---
results_df = pd.DataFrame(results_summary).round(3)
print("\n=== Summary Results ===")
display(results_df)



--- Training on Hourly | target=N2O_Flux | predictors=base ---
Train period: 0 → 10190
Test  period: 10191 → 15850

Model evaluation:
  R² (linear scale): -0.014
  Spearman ρ:        0.413

--- Training on Hourly | target=N2O_Flux | predictors=lag ---
Train period: 0 → 2837
Test  period: 2838 → 15850

Model evaluation:
  R² (linear scale): -0.100
  Spearman ρ:        0.318

--- Training on Hourly | target=N2O_Flux | predictors=main ---
Train period: 0 → 10746
Test  period: 10747 → 15850

Model evaluation:
  R² (linear scale): 0.034
  Spearman ρ:        0.371

--- Training on Hourly | target=N2O_Flux_ln | predictors=base ---
Train period: 0 → 10190
Test  period: 10191 → 15850

Model evaluation:
  R² (linear scale): 0.075
  Spearman ρ:        0.396

--- Training on Hourly | target=N2O_Flux_ln | predictors=lag ---
Train period: 0 → 2837
Test  period: 2838 → 15850

Model evaluation:
  R² (linear scale): 0.017
  Spearman ρ:        0.335

--- Training on Hourly | target=N2O_Flux_ln | predic

,Dataset,Target,Predictor_Set,R²,Spearman_ρ
0,Hourly,N2O_Flux,base,-0.014,0.413
1,Hourly,N2O_Flux,lag,-0.100,0.318
2,Hourly,N2O_Flux,main,0.034,0.371
3,Hourly,N2O_Flux_ln,base,0.075,0.396
4,Hourly,N2O_Flux_ln,lag,0.017,0.335
5,Hourly,N2O_Flux_ln,main,0.086,0.335
6,Hourly A,N2O_Flux,base,0.060,0.404
7,Hourly A,N2O_Flux,lag,0.038,0.365
8,Hourly A,N2O_Flux,main,0.120,0.343
9,Hourly A,N2O_Flux_ln,base,0.078,0.373


In [15]:
from sklearn.linear_model import ElasticNetCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score
from scipy.stats import spearmanr
import numpy as np

def train_elasticnet_timeseries(df, predictors, target, n_splits=5):
    # Drop rows with missing values in predictors or target
    data = df.dropna(subset=predictors + [target]).copy()

    X = data[predictors].values
    y = data[target].values

    # Time-series aware CV (no shuffling)
    tscv = TimeSeriesSplit(n_splits=n_splits)

    # Model pipeline: scale → ElasticNet (with CV tuning)
    model = Pipeline([
        ("scaler", StandardScaler()),
        ("enet", ElasticNetCV(
            l1_ratio=[0.1, 0.3, 0.5, 0.7, 0.9, 1.0],  # search mix between L1/L2
            alphas=np.logspace(-3, 2, 50),          # log grid of penalty strengths
            cv=tscv,
            max_iter=5000
        ))
    ])

    model.fit(X, y)
    y_pred = model.predict(X)

    return {
        "model": model,
        "r2": r2_score(y, y_pred),
        "spearman_rho": spearmanr(y, y_pred).correlation
    }


In [20]:
results_enet_summary = []

for df_name, df in datasets.items():
    for set_name, predictors in predictor_sets.items():
        print(f"\n--- Elastic Net | Dataset={df_name} | predictors={set_name} ---")

        try:
            res = train_elasticnet_timeseries(
                df=df,
                predictors=predictors,
                target="N2O_Flux_ln",   # ALWAYS log target
                n_splits=5              # time-series split
            )

            results_enet_summary.append({
                "Dataset": df_name,
                "Predictor_Set": set_name,
                "Target": "N2O_Flux_ln",
                "R²": round(res["r2"], 3),
                "Spearman_ρ": round(res["spearman_rho"], 3),
                "Best Alpha": res["model"].named_steps["enet"].alpha_,
                "Best l1_ratio": res["model"].named_steps["enet"].l1_ratio_
            })

        except Exception as e:
            print(f"Skipped {df_name} | {set_name} due to error: {e}")
            results_enet_summary.append({
                "Dataset": df_name,
                "Predictor_Set": set_name,
                "Target": "N2O_Flux_ln",
                "R²": None,
                "Spearman_ρ": None,
                "Best Alpha": None,
                "Best l1_ratio": None
            })

# Convert to DataFrame
results_enet_df = pd.DataFrame(results_enet_summary)
display(results_enet_df.sort_values(
            by=["R²", "Spearman_ρ"],
            ascending=[False, False]))



--- Elastic Net | Dataset=Hourly | predictors=base ---

--- Elastic Net | Dataset=Hourly | predictors=lag ---

--- Elastic Net | Dataset=Hourly | predictors=main ---

--- Elastic Net | Dataset=Hourly A | predictors=base ---

--- Elastic Net | Dataset=Hourly A | predictors=lag ---

--- Elastic Net | Dataset=Hourly A | predictors=main ---

--- Elastic Net | Dataset=Hourly B | predictors=base ---

--- Elastic Net | Dataset=Hourly B | predictors=lag ---

--- Elastic Net | Dataset=Hourly B | predictors=main ---

--- Elastic Net | Dataset=Daily A | predictors=base ---

--- Elastic Net | Dataset=Daily A | predictors=lag ---


/home/anding/miniconda3/envs/dsl/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.00836379744952609, tolerance: 0.005003068416257733
  model = cd_fast.enet_coordinate_descent_gram(



--- Elastic Net | Dataset=Daily A | predictors=main ---

--- Elastic Net | Dataset=Daily B | predictors=base ---

--- Elastic Net | Dataset=Daily B | predictors=lag ---

--- Elastic Net | Dataset=Daily B | predictors=main ---

--- Elastic Net | Dataset=Daily | predictors=base ---

--- Elastic Net | Dataset=Daily | predictors=lag ---

--- Elastic Net | Dataset=Daily | predictors=main ---


,Dataset,Predictor_Set,Target,R²,Spearman_ρ,Best Alpha,Best l1_ratio
1,Hourly,lag,N2O_Flux_ln,0.307,0.442,0.109854,0.1
13,Daily B,lag,N2O_Flux_ln,0.304,0.450,0.004095,0.3
16,Daily,lag,N2O_Flux_ln,0.301,0.459,0.013257,0.1
7,Hourly B,lag,N2O_Flux_ln,0.295,0.422,0.175751,0.1
9,Daily A,base,N2O_Flux_ln,0.265,0.439,0.138950,0.1
6,Hourly B,base,N2O_Flux_ln,0.264,0.404,0.054287,0.1
3,Hourly A,base,N2O_Flux_ln,0.263,0.479,0.175751,0.1
10,Daily A,lag,N2O_Flux_ln,0.263,0.463,0.138950,0.3
0,Hourly,base,N2O_Flux_ln,0.252,0.410,0.068665,0.1
15,Daily,base,N2O_Flux_ln,0.244,0.408,0.026827,0.1


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV

# Time-series split
tscv = TimeSeriesSplit(n_splits=5)

# Base model
rf = RandomForestRegressor(random_state=42, n_jobs=-1)

# Reasonable search space for environmental data
param_grid = {
    "n_estimators": [100, 300, 600],
    "max_depth": [10, 20, 30, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2", 0.5],   # subset of features per split
    "bootstrap": [True, False]
}

grid_search = GridSearchCV(
    rf,
    param_grid=param_grid,
    cv=tscv,
    scoring="r2",
    n_jobs=-1,
    verbose=2
)

X = chamau_daily[predictor_sets["lag"]].dropna()
y = chamau_daily.loc[X.index, "N2O_Flux_ln"]

grid_search.fit(X, y)
print("Best Params:", grid_search.best_params_)
print("Best CV R²:", grid_search.best_score_)
